In [1]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
from nwbwidgets.allen import AllenRasterWidget
from nwbwidgets.allen import AllenPSTHWidget
import pandas as pd
import numpy as np
import sys
import os

# Add parent directory to the system path
sys.path.append(os.path.abspath(".."))
from helpers import *
from generate import *
from pathlib import Path
from plot import *

io = NWBHDF5IO('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Mice_data/context/AB122_20240804_134554.nwb')
nwbfile = io.read()


""" 
# Load the file with the mice data
nwbfiles = []

paths = ['/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', 
        '/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB087_20231017_141901.nwb']
mouse_names = ['AB077_20230531_143839', 'AB087_20231017_141901']

for path in paths:
    io = NWBHDF5IO(path)
    nwbfiles.append(io.read())

io = NWBHDF5IO('/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839', mode='r')
nwbfile = io.read()
# To do : create a list of mouse names, to be used for the .py file.
mouse_name = 'AB077_20230531_143839'
"""

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)  # Show all columns


/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


### Loading the tables and preprocessing


#### For each line of trials table, we start by testing 10 neurons. In other words, if there is a stimulus (whisker or auditory) at a certain start time then, compute the number of spikes for each trial for each neuron.

In [15]:
units = nwbfile.units.to_dataframe()
trials = nwbfile.trials.to_dataframe()

# Preprocessing s
filtered_units = units[(units['bc_label'] == 'good') & (units['ccf_acronym'].str.contains('[A-Z]'))]


# Some chosen columns
#cons_columns  = ["cluster_id", "firing_rate", "ccf_acronym", "ccf_name", "ccf_parent_acronym", "ccf_parent_name", "spike_times"]#, "lick_flag", "lick_time"]
#filtered_units = filtered_units[cons_columns]

In [112]:
trials[trials['context']=='passive']

,start_time,stop_time,trial_type,perf,whisker_stim,whisker_stim_amplitude,whisker_stim_duration,whisker_stim_time,auditory_stim,auditory_stim_amplitude,auditory_stim_frequency,auditory_stim_duration,auditory_stim_time,no_stim,no_stim_time,stim_onset,reward_available,response_window_start_time,response_window_stop_time,lick_flag,lick_time,abort_window_start_time,abort_window_stop_time,early_lick,context,context_background,opto_stim,opto_grid_ap,opto_grid_ml,opto_grid_no,opto_stim_start_time,opto_stim_stop_time,opto_stim_amplitude,opto_stim_frequency,trial_id
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,22.705635,23.805635,auditory_trial,6,0,0.0,0,NaN,1,10,10000,0,22.705635,0,NaN,22.705635,1,22.805635,23.805635,0,NaN,22.705635,22.705635,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,25.823920,26.923920,whisker_trial,6,1,3.2,3,25.823920,0,0,0,0,NaN,0,NaN,25.823920,1,25.923920,26.923920,0,NaN,25.823920,25.823920,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,28.856292,29.956292,whisker_trial,6,1,3.2,3,28.856292,0,0,0,0,NaN,0,NaN,28.856292,1,28.956292,29.956292,0,NaN,28.856292,28.856292,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31.925298,33.025298,whisker_trial,6,1,3.2,3,31.925298,0,0,0,0,NaN,0,NaN,31.925298,1,32.025298,33.025298,0,NaN,31.925298,31.925298,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,34.998745,36.098745,whisker_trial,6,1,3.2,3,34.998745,0,0,0,0,NaN,0,NaN,34.998745,1,35.098745,36.098745,0,NaN,34.998745,34.998745,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,4005.387284,4006.487284,whisker_trial,6,1,3.2,3,4005.387284,0,0,0,0,NaN,0,NaN,4005.387284,1,4005.487284,4006.487284,0,NaN,4005.387284,4005.387284,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,427
428,4008.430656,4009.530656,whisker_trial,6,1,3.2,3,4008.430656,0,0,0,0,NaN,0,NaN,4008.430656,1,4008.530656,4009.530656,0,NaN,4008.430656,4008.430656,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,428
429,4011.488732,4012.588732,whisker_trial,6,1,3.2,3,4011.488732,0,0,0,0,NaN,0,NaN,4011.488732,1,4011.588732,4012.588732,0,NaN,4011.488732,4011.488732,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429


### Whisker &  Stimulation
#### Now iterate in each row of short\_units and create a column with tables to append values for number of spikes for each 

In [6]:
### part 1 : with passive trials
#active_trials = trials[trials['context']] == 'passive'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
### part 2 : with active trials
#passive_trials = trials[trials['context']] == 'active'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
# Create a new dataframe 
mouse_name = "AB082_20230630_101353.nwb"

data['pre'] = 0.2
data['post'] = 0.2
data['Mouse name'] = mouse_name



folder = f'Data/{mouse_name}'
os.makedirs(folder, exist_ok=True)

#### Only problem : arrays present in columns are converted to string of array, so careful while preprocessing ####

###### It is better to use parquet files ######
#data.to_csv(f'{mouse_name}_Selectivity_Dataframe.csv', index=True, header=True)
#data.to_pickle(f'{mouse_name}_Selectivity_Dataframe.pkl')
data.to_parquet(f'{folder}/{mouse_name}_Selectivity_Dataframe.parquet', index=False)